In [2]:
import subprocess
import json
import os

# perf and program parameters
scene_indices = range(0, 3)
epochs = 5
path_perf = "/usr/lib/linux-tools-6.8.0-88/perf"
fpath_json = os.path.abspath("/tmp/tracer-perf-%d.json")
path_program = os.path.abspath("./build/tracer/Release/tracer")

In [ ]:
subprocess.run(['./benchmark.sh'], cwd='./tracer/')

# run it!
for i_scene in scene_indices:
    subprocess.run([
        path_perf, "stat", 
        "-j", '-o', fpath_json % i_scene, 
        "-r", str(epochs), 
        '-e', 'cycles,instructions,cache-references,cache-misses,' 
                + 'L1-dcache-loads,L1-dcache-load-misses,LLC-loads,LLC-load-misses,'
                + 'branches,branch-misses,stalled-cycles-frontend,stalled-cycles-backend,'
                + 'dTLB-loads,dTLB-load-misses',
        '--', 
        path_program, str(i_scene), '0' # don't create output image
        ],
    )

Build
-- Configuring done (0.1s)
-- Generating done (0.0s)
-- Build files have been written to: /home/nick/git/weekend-raytracer/build
[1/2] Building CXX object tracer/CMakeFiles/tracer.dir/Release/main.cpp.o
[2/2] Linking CXX executable tracer/Release/tracer



Rendered scene 0 in 38.563s (0.428478ms/px)                       
Rendering scene 0...

In [71]:
# load perf output
result = { }
path_json = fpath_json % 0
with open(path_json, 'r') as file:
    for line in file.readlines():
        event_raw = json.loads(line)
        result[event_raw['event']] = event_raw
result 

for name, data in result.items():
    print(f"{name}: {data['counter-value']}{data['unit']} ({data['metric-value']} {data['metric-unit']})")

task-clock: 35102.405875msec (0.999836 CPUs utilized)
context-switches: 308.000000 (8.774327 /sec)
cpu-migrations: 7.000000 (0.199417 /sec)
page-faults: 684.000000 (19.485844 /sec)
cycles: 125367661740.000000 (3.571483 GHz)
instructions: 443367088141.000000 (3.536535 insn per cycle)
branches: 37157798483.000000 (1.058554 G/sec)
branch-misses: 88322197.000000 (0.237695 of all branches)
